In [11]:
import math
import random

# this methode finds the indices for all occurencys of target in data 
def findAll(data: bytes, target: int) -> list:
    result = []
    last_index = 0
    while last_index != -1:
        i = data[last_index:].find(target)
        if(i < 0):
            return result
        else:
            result += [last_index+i]
            last_index = last_index+i+1
            
# this methode finds the indices for all occurencys of a word in data 
def findKeyword(data, word):
    word_array = word.encode('ascii')
    result = []
    possible_indices = findAll(data, word_array[0])
    for p in possible_indices:
        same = True
        for i in range(1, len(word)):
            if(data[p+i] != word_array[i]):
                same = False
        if(same):
            result += [p]
    return result

def distance(a, b):
    return math.sqrt(pow(a[0]-b[0],2)+pow(a[1]-b[1],2)+pow(a[2]-b[2],2))

In [12]:
def manipulateVoxel(voxel):
    centers = [[random.randrange(0,21),random.randrange(0,21),random.randrange(0,21),7+i] for i in range(0,100)]
    cluster = [[] for i in range(0,len(centers))]
    for v in voxel:
        center_id = 0
        col = 80
        dist = 1000000;
        for i in range(0,len(centers)):
            d = distance(centers[i][0:3],v[0:3])
            if(d<dist):
                dist = d
                col = centers[i][3]
            v[3] = col
            cluster[i] += [v]
    return cluster # not really necessary

In [14]:
def load(name: str) -> bytes:
    binary_file = open(name, 'rb')
    data = binary_file.read()
    binary_file.close()
    return data

def save(name: str):
    f = open(name, 'wb+')
    f.write(result)
    f.close()  

In [49]:
#SIZE\0c 00 00 00 \ 00 00 00 00 \ 03 00 00 00 \ 03 00 00 00 \ 03 00 00 00 \ XYZI \54 00 00 00 \ 00 00 00 00 \ 14 00 00 00'\
# id                                 size x       size y         size z      id                                 # voxel   

# |SIZE|#bytes|#children|size x|size y|size z|XYZI|#bytes|#childeren|#voxel n|[x y z c]|...|[x y z c]|
# ^start_id                                  ^xyz_id                         ^vox_id    ^[x y z c]*n ^end_id

def toBytes(a: int) -> bytes:
    return a.to_bytes(4, byteorder="little")

# takes data in for [[a_1,a_2,...],[b_1,b_2,...],...] and turns it into a byte array (a_n,b_n,... <256)
def toBytearray(data) -> bytes:
    byte = bytearray()
    for d in data:
        for i in d:
            byte += (i).to_bytes(1, byteorder="little")
    return byte

def sizeChunk(x: int, y: int, z: int) -> bytes:
    result = bytearray()
    result.extend(map(ord, "SIZE"))
    result += toBytes(12) + toBytes(0) + toBytes(x) + toBytes(y) + toBytes(z)
    
    return result

def xyziChunk(voxel) -> bytes:
    n = len(voxel)
    result = bytearray()
    result.extend(map(ord, "XYZI"))
    result += toBytes(4+n*4) + toBytes(0) + toBytes(n) + toBytearray(voxel)
    
    return result

def maxDimensions(voxel):
    result = [0,0,0]
    for v in voxel:
        for i in range(0, len(result)):
            result[i] = max(result[i], v[i])
            
    return result

def modelData(voxel) -> bytes:
    result = bytearray()
    dimensions = maxDimensions(voxel)
    result += sizeChunk(dimensions[0],dimensions[1],dimensions[2])
    result += xyziChunk(voxel)
    return result

modelData([[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]])

bytearray(b'SIZE\x0c\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x04\x00\x00\x00\x04\x00\x00\x00XYZI\x14\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x01\x01\x01\x01\x02\x02\x02\x02\x03\x03\x03\x03\x04\x04\x04\x04')

In [19]:

def workOnModel(data, start_id):
    #calculate section start ids and length
    xyz_id = start_id+24
    vox_id = xyz_id+16
    n = int.from_bytes(data[xyz_id+12:vox_id], byteorder = "little")
    end_id = vox_id + n*4
    
    print(data[start_id:vox_id])
    #seperate header from voxel
    size_header = data[start_id:xyz_id]
    xyzi_header = data[xyz_id:vox_id]
    voxel = [[data[vox_id+i*4], data[vox_id+1+i*4], data[vox_id+2+i*4], data[vox_id+3+i*4]] for i in range(0, n)]
    
    ##manipulate voxel
    cluster = manipulateVoxel(voxel)
    
    #result = 
   
    
    return size_header + xyzi_header + toBytearray(voxel), end_id

In [20]:
# Read the whole file at once
data = load("3x3x3.vox")
model_ids = findKeyword(data, 'SIZE') 

header = data[:model_ids[0]]

model_bytes = bytearray()
end_id=0
for i in model_ids:
    bytes_out, end_id = workOnModel(data, i)
    model_bytes += bytes_out

#seperate header and tail, read number of voxels n
tail = data[end_id:]
    
#calculate endresult
result = header+model_bytes+tail;
    
save("resultColor.vox")
print('done')

b'SIZE\x0c\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00XYZIT\x00\x00\x00\x00\x00\x00\x00\x14\x00\x00\x00'
done
